# TV Script Generation
In this project, you'll generate your own [Simpsons](https://en.wikipedia.org/wiki/The_Simpsons) TV scripts using RNNs.  You'll be using part of the [Simpsons dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data) of scripts from 27 seasons.  The Neural Network you'll build will generate a new TV script for a scene at [Moe's Tavern](https://simpsonswiki.com/wiki/Moe's_Tavern).
## Get the Data
The data is already provided for you.  You'll be using a subset of the original dataset.  It consists of only the scenes in Moe's Tavern.  This doesn't include other versions of the tavern, like "Moe's Cavern", "Flaming Moe's", "Uncle Moe's Family Feed-Bag", etc..

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper

data_dir = './data/simpsons/moes_tavern_lines.txt'
text = helper.load_data(data_dir)
# Ignore notice, since we don't use it for analysing the data
text = text[81:]

In [2]:
print('text type: {}'.format(type(text)))
print('text: \n\n{}\n'.format(text[:100]))
scenes = text.split('\n\n')
print('scenes: \n\n{}\n'.format(scenes[0]))

text.split()

text type: <class 'str'>
text: 

Moe_Szyslak: (INTO PHONE) Moe's Tavern. Where the elite meet to drink.
Bart_Simpson: Eh, yeah, hello

scenes: 

Moe_Szyslak: (INTO PHONE) Moe's Tavern. Where the elite meet to drink.
Bart_Simpson: Eh, yeah, hello, is Mike there? Last name, Rotch.
Moe_Szyslak: (INTO PHONE) Hold on, I'll check. (TO BARFLIES) Mike Rotch. Mike Rotch. Hey, has anybody seen Mike Rotch, lately?
Moe_Szyslak: (INTO PHONE) Listen you little puke. One of these days I'm gonna catch you, and I'm gonna carve my name on your back with an ice pick.
Moe_Szyslak: What's the matter Homer? You're not your normal effervescent self.
Homer_Simpson: I got my problems, Moe. Give me another one.
Moe_Szyslak: Homer, hey, you should not drink to forget your problems.
Barney_Gumble: Yeah, you should only drink to enhance your social skills.



['Moe_Szyslak:',
 '(INTO',
 'PHONE)',
 "Moe's",
 'Tavern.',
 'Where',
 'the',
 'elite',
 'meet',
 'to',
 'drink.',
 'Bart_Simpson:',
 'Eh,',
 'yeah,',
 'hello,',
 'is',
 'Mike',
 'there?',
 'Last',
 'name,',
 'Rotch.',
 'Moe_Szyslak:',
 '(INTO',
 'PHONE)',
 'Hold',
 'on,',
 "I'll",
 'check.',
 '(TO',
 'BARFLIES)',
 'Mike',
 'Rotch.',
 'Mike',
 'Rotch.',
 'Hey,',
 'has',
 'anybody',
 'seen',
 'Mike',
 'Rotch,',
 'lately?',
 'Moe_Szyslak:',
 '(INTO',
 'PHONE)',
 'Listen',
 'you',
 'little',
 'puke.',
 'One',
 'of',
 'these',
 'days',
 "I'm",
 'gonna',
 'catch',
 'you,',
 'and',
 "I'm",
 'gonna',
 'carve',
 'my',
 'name',
 'on',
 'your',
 'back',
 'with',
 'an',
 'ice',
 'pick.',
 'Moe_Szyslak:',
 "What's",
 'the',
 'matter',
 'Homer?',
 "You're",
 'not',
 'your',
 'normal',
 'effervescent',
 'self.',
 'Homer_Simpson:',
 'I',
 'got',
 'my',
 'problems,',
 'Moe.',
 'Give',
 'me',
 'another',
 'one.',
 'Moe_Szyslak:',
 'Homer,',
 'hey,',
 'you',
 'should',
 'not',
 'drink',
 'to',
 'forget'

## Explore the Data
Play around with `view_sentence_range` to view different parts of the data.

In [3]:
view_sentence_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))
scenes = text.split('\n\n')
print('Number of scenes: {}'.format(len(scenes)))
sentence_count_scene = [scene.count('\n') for scene in scenes]
print('Average number of sentences in each scene: {}'.format(np.average(sentence_count_scene)))

sentences = [sentence for scene in scenes for sentence in scene.split('\n')]
print('Number of lines: {}'.format(len(sentences)))
word_count_sentence = [len(sentence.split()) for sentence in sentences]
print('Average number of words in each line: {}'.format(np.average(word_count_sentence)))

print()
print('The sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 11492
Number of scenes: 262
Average number of sentences in each scene: 15.248091603053435
Number of lines: 4257
Average number of words in each line: 11.50434578341555

The sentences 0 to 10:
Moe_Szyslak: (INTO PHONE) Moe's Tavern. Where the elite meet to drink.
Bart_Simpson: Eh, yeah, hello, is Mike there? Last name, Rotch.
Moe_Szyslak: (INTO PHONE) Hold on, I'll check. (TO BARFLIES) Mike Rotch. Mike Rotch. Hey, has anybody seen Mike Rotch, lately?
Moe_Szyslak: (INTO PHONE) Listen you little puke. One of these days I'm gonna catch you, and I'm gonna carve my name on your back with an ice pick.
Moe_Szyslak: What's the matter Homer? You're not your normal effervescent self.
Homer_Simpson: I got my problems, Moe. Give me another one.
Moe_Szyslak: Homer, hey, you should not drink to forget your problems.
Barney_Gumble: Yeah, you should only drink to enhance your social skills.




## Implement Preprocessing Functions
The first thing to do to any dataset is preprocessing.  Implement the following preprocessing functions below:
- Lookup Table
- Tokenize Punctuation

### Lookup Table
To create a word embedding, you first need to transform the words to ids.  In this function, create two dictionaries:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

Return these dictionaries in the following tuple `(vocab_to_int, int_to_vocab)`

In [4]:
import numpy as np
import problem_unittests as tests
from collections import Counter

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    # remove all punctutation
    counts = Counter(text)
    vocab = sorted(counts, key=counts.get, reverse=True)
    vocab_to_int = {word: ii for ii, word in enumerate(vocab)}
    int_to_vocab = dict([(v, k) for k, v in vocab_to_int.items()])
    # print('vocab_to_int size: {}'.format(len(vocab_to_int)))
    # print('int_to_vocab size: {}'.format(len(int_to_vocab)))
    # TODO: Implement Function
    return vocab_to_int, int_to_vocab


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_create_lookup_tables(create_lookup_tables)

/usr/local/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


Tests Passed


### Tokenize Punctuation
We'll be splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks make it hard for the neural network to distinguish between the word "bye" and "bye!".

Implement the function `token_lookup` to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||".  Create a dictionary for the following symbols where the symbol is the key and value is the token:
- Period ( . )
- Comma ( , )
- Quotation Mark ( " )
- Semicolon ( ; )
- Exclamation mark ( ! )
- Question mark ( ? )
- Left Parentheses ( ( )
- Right Parentheses ( ) )
- Dash ( -- )
- Return ( \n )

This dictionary will be used to token the symbols and add the delimiter (space) around it.  This separates the symbols as it's own word, making it easier for the neural network to predict on the next word. Make sure you don't use a token that could be confused as a word. Instead of using the token "dash", try using something like "||dash||".

In [5]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenize dictionary where the key is the punctuation and the value is the token
    """
    # TODO: Implement Function
    puncs = ['.', ',', '"', ';', '!', '?', '(', ')', '--', '\n']
    tokens = ['Period', 'Comma', 'Quotation_Mark', 'Semicolon', 'Exclamation_mark', 'Question_mark', 'Left_Parentheses', 'Right_Parentheses', 'Dash', 'Return']
    punc_to_token = {punc: "||"+token+"||"for punc, token in zip(puncs, tokens)}
    return punc_to_token

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_tokenize(token_lookup)

Tests Passed


## Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [6]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Preprocess Training, Validation, and Testing Data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [7]:
int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

In [8]:
print('show data:')
print('int_text: {}'.format(int_text[:100]))

show data:
int_text: [9, 3, 121, 172, 4, 173, 332, 0, 137, 5, 2896, 513, 13, 145, 0, 1, 146, 174, 2, 36, 2, 212, 2, 22, 961, 79, 10, 203, 198, 2, 1170, 0, 1, 9, 3, 121, 172, 4, 379, 28, 2, 94, 267, 0, 3, 13, 604, 4, 961, 1170, 0, 961, 1170, 0, 29, 2, 175, 664, 299, 961, 1170, 2, 1440, 10, 1, 9, 3, 121, 172, 4, 220, 7, 87, 1441, 0, 60, 15, 185, 729, 24, 78, 962, 7, 2, 14, 24, 78, 1900, 16, 198, 28, 26, 72, 35, 84, 841, 423, 0, 1, 9]


## Build the Neural Network
You'll build the components necessary to build a RNN by implementing the following functions below:
- get_inputs
- get_init_cell
- get_embed
- build_rnn
- build_nn
- get_batches

### Check the Version of TensorFlow and Access to GPU

In [9]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.4.0
Default GPU Device: /device:GPU:0


### Input
Implement the `get_inputs()` function to create TF Placeholders for the Neural Network.  It should create the following placeholders:
- Input text placeholder named "input" using the [TF Placeholder](https://www.tensorflow.org/api_docs/python/tf/placeholder) `name` parameter.
- Targets placeholder
- Learning Rate placeholder

Return the placeholders in the following tuple `(Input, Targets, LearningRate)`

In [10]:
def get_inputs():
    """
    Create TF Placeholders for input, targets, and learning rate.
    :return: Tuple (input, targets, learning rate)
    """
    # TODO: Implement Function
    _input = tf.placeholder(dtype=tf.int32, shape=(None, None), name='input')
    _targets = tf.placeholder(dtype=tf.int32, shape=(None, None), name='targets')
    _learning_rate = tf.placeholder(dtype=tf.float32, name='lr')
    return _input, _targets, _learning_rate


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_inputs(get_inputs)

Tests Passed


### Build RNN Cell and Initialize
Stack one or more [`BasicLSTMCells`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/BasicLSTMCell) in a [`MultiRNNCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell).
- The Rnn size should be set using `rnn_size`
- Initalize Cell State using the MultiRNNCell's [`zero_state()`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell#zero_state) function
    - Apply the name "initial_state" to the initial state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the cell and initial state in the following tuple `(Cell, InitialState)`

In [11]:
def get_init_cell(batch_size, rnn_size):
    """
    Create an RNN Cell and initialize it.
    :param batch_size: Size of batches
    :param rnn_size: Size of RNNs
    :return: Tuple (cell, initialize state)
    """
    # TODO: Implement Function
    def lstm_cell(rnn_size):
        cell = tf.contrib.rnn.BasicLSTMCell(rnn_size)
        return cell
    cell = tf.contrib.rnn.MultiRNNCell([lstm_cell(rnn_size) for _ in range(2)])
    initial_state = cell.zero_state(batch_size, tf.float32)
    initial_state = tf.identity(initial_state, name='initial_state')
    return cell, initial_state


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_init_cell(get_init_cell)

Tests Passed


### Word Embedding
Apply embedding to `input_data` using TensorFlow.  Return the embedded sequence.

In [12]:
def get_embed(input_data, vocab_size, embed_dim):
    """
    Create embedding for <input_data>.
    :param input_data: TF placeholder for text input.
    :param vocab_size: Number of words in vocabulary.
    :param embed_dim: Number of embedding dimensions
    :return: Embedded input.
    """
    # TODO: Implement Function
    embedding = tf.Variable(tf.random_uniform(shape=(vocab_size, embed_dim), minval=-1, maxval=1))
    embed = tf.nn.embedding_lookup(embedding, input_data)
    return embed


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_embed(get_embed)

Tests Passed


### Build RNN
You created a RNN Cell in the `get_init_cell()` function.  Time to use the cell to create a RNN.
- Build the RNN using the [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)
 - Apply the name "final_state" to the final state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the outputs and final_state state in the following tuple `(Outputs, FinalState)` 

In [13]:
def build_rnn(cell, inputs):
    """
    Create a RNN using a RNN Cell
    :param cell: RNN Cell
    :param inputs: Input text data
    :return: Tuple (Outputs, Final State)
    """
    # TODO: Implement Function
    outputs, final_state = tf.nn.dynamic_rnn(cell, inputs, dtype=tf.float32)
    final_state = tf.identity(final_state, 'final_state')
    return outputs, final_state


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_rnn(build_rnn)

Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:
- Apply embedding to `input_data` using your `get_embed(input_data, vocab_size, embed_dim)` function.
- Build RNN using `cell` and your `build_rnn(cell, inputs)` function.
- Apply a fully connected layer with a linear activation and `vocab_size` as the number of outputs.

Return the logits and final state in the following tuple (Logits, FinalState) 

In [14]:
def build_nn(cell, rnn_size, input_data, vocab_size, embed_dim):
    """
    Build part of the neural network
    :param cell: RNN cell
    :param rnn_size: Size of rnns
    :param input_data: Input data
    :param vocab_size: Vocabulary size
    :param embed_dim: Number of embedding dimensions
    :return: Tuple (Logits, FinalState)
    """
    # TODO: Implement Function
    embeddings = get_embed(input_data, vocab_size, embed_dim)
    outputs, final_state = build_rnn(cell, embeddings)
    logits = tf.contrib.layers.fully_connected(outputs, vocab_size, activation_fn=None)
    return logits, final_state


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_nn(build_nn)

Tests Passed


### Batches
Implement `get_batches` to create batches of input and targets using `int_text`.  The batches should be a Numpy array with the shape `(number of batches, 2, batch size, sequence length)`. Each batch contains two elements:
- The first element is a single batch of **input** with the shape `[batch size, sequence length]`
- The second element is a single batch of **targets** with the shape `[batch size, sequence length]`

If you can't fill the last batch with enough data, drop the last batch.

For exmple, `get_batches([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], 3, 2)` would return a Numpy array of the following:
```
[
  # First Batch
  [
    # Batch of Input
    [[ 1  2], [ 7  8], [13 14]]
    # Batch of targets
    [[ 2  3], [ 8  9], [14 15]]
  ]

  # Second Batch
  [
    # Batch of Input
    [[ 3  4], [ 9 10], [15 16]]
    # Batch of targets
    [[ 4  5], [10 11], [16 17]]
  ]

  # Third Batch
  [
    # Batch of Input
    [[ 5  6], [11 12], [17 18]]
    # Batch of targets
    [[ 6  7], [12 13], [18  1]]
  ]
]
```

Notice that the last target value in the last batch is the first input value of the first batch. In this case, `1`. This is a common technique used when creating sequence batches, although it is rather unintuitive.

In [15]:
def get_batches(int_text, batch_size, seq_length):
    """
    Return batches of input and target
    :param int_text: Text with the words replaced by their ids
    :param batch_size: The size of batch
    :param seq_length: The length of sequence
    :return: Batches as a Numpy array
    """
    # TODO: Implement Function
    batch_ints = batch_size * seq_length
    num_batches = len(int_text) // batch_ints
    int_txt_cut = int_text[:num_batches * batch_ints]
    batches = []
    for i in range(num_batches):
        x = []
        y = []
        for j in range(batch_size):
            tmp = int_txt_cut[j*seq_length*num_batches+i*seq_length : j*seq_length*num_batches+(i+1)*seq_length]
            x.append(tmp)
            if i == num_batches - 1 and j == batch_size -1:
                tmp[:-1] = int_txt_cut[j*seq_length*num_batches+i*seq_length+1: j*seq_length*num_batches+(i+1)*seq_length]
                tmp[-1] = int_txt_cut[0]
            else:
                tmp = int_txt_cut[j*seq_length*num_batches+i*seq_length+1 : j*seq_length*num_batches+(i+1)*seq_length+1]
            y.append(tmp)
        batches.append([x, y])
    return np.array(batches)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_batches(get_batches)

Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `num_epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `embed_dim` to the size of the embedding.
- Set `seq_length` to the length of sequence.
- Set `learning_rate` to the learning rate.
- Set `show_every_n_batches` to the number of batches the neural network should print progress.

In [24]:
# Number of Epochs
num_epochs = 300
# Batch Size
batch_size = 128
# RNN Size
rnn_size = 512
# Embedding Dimension Size
embed_dim = 256
# Sequence Length
seq_length = 16
# Learning Rate
learning_rate = 0.001
# Show stats for every n number of batches
show_every_n_batches = 2

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
save_dir = './save'

### Build the Graph
Build the graph using the neural network you implemented.

In [25]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from tensorflow.contrib import seq2seq

train_graph = tf.Graph()
with train_graph.as_default():
    vocab_size = len(int_to_vocab)
    input_text, targets, lr = get_inputs()
    input_data_shape = tf.shape(input_text)
    cell, initial_state = get_init_cell(input_data_shape[0], rnn_size)
    logits, final_state = build_nn(cell, rnn_size, input_text, vocab_size, embed_dim)

    # Probabilities for generating words
    probs = tf.nn.softmax(logits, name='probs')

    # Loss function
    cost = seq2seq.sequence_loss(
        logits,
        targets,
        tf.ones([input_data_shape[0], input_data_shape[1]]))

    # Optimizer
    optimizer = tf.train.AdamOptimizer(lr)

    # Gradient Clipping
    gradients = optimizer.compute_gradients(cost)
    capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
    train_op = optimizer.apply_gradients(capped_gradients)

## Train
Train the neural network on the preprocessed data.  If you have a hard time getting a good loss, check the [forums](https://discussions.udacity.com/) to see if anyone is having the same problem.

In [26]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
batches = get_batches(int_text, batch_size, seq_length)

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(num_epochs):
        state = sess.run(initial_state, {input_text: batches[0][0]})

        for batch_i, (x, y) in enumerate(batches):
            feed = {
                input_text: x,
                targets: y,
                initial_state: state,
                lr: learning_rate}
            train_loss, state, _ = sess.run([cost, final_state, train_op], feed)

            # Show every <show_every_n_batches> batches
            if (epoch_i * len(batches) + batch_i) % show_every_n_batches == 0:
                print('Epoch {:>3} Batch {:>4}/{}   train_loss = {:.3f}'.format(
                    epoch_i,
                    batch_i,
                    len(batches),
                    train_loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_dir)
    print('Model Trained and Saved')

Epoch   0 Batch    0/33   train_loss = 8.823
Epoch   0 Batch    2/33   train_loss = 8.722
Epoch   0 Batch    4/33   train_loss = 8.125
Epoch   0 Batch    6/33   train_loss = 7.169
Epoch   0 Batch    8/33   train_loss = 6.634
Epoch   0 Batch   10/33   train_loss = 6.583
Epoch   0 Batch   12/33   train_loss = 6.543
Epoch   0 Batch   14/33   train_loss = 6.720
Epoch   0 Batch   16/33   train_loss = 6.526
Epoch   0 Batch   18/33   train_loss = 6.564
Epoch   0 Batch   20/33   train_loss = 6.503
Epoch   0 Batch   22/33   train_loss = 6.661
Epoch   0 Batch   24/33   train_loss = 6.467
Epoch   0 Batch   26/33   train_loss = 6.449
Epoch   0 Batch   28/33   train_loss = 6.505
Epoch   0 Batch   30/33   train_loss = 6.496
Epoch   0 Batch   32/33   train_loss = 6.688
Epoch   1 Batch    1/33   train_loss = 6.306
Epoch   1 Batch    3/33   train_loss = 6.270
Epoch   1 Batch    5/33   train_loss = 6.324
Epoch   1 Batch    7/33   train_loss = 6.198
Epoch   1 Batch    9/33   train_loss = 6.082
Epoch   1 

Epoch  11 Batch    3/33   train_loss = 5.407
Epoch  11 Batch    5/33   train_loss = 5.618
Epoch  11 Batch    7/33   train_loss = 5.531
Epoch  11 Batch    9/33   train_loss = 5.427
Epoch  11 Batch   11/33   train_loss = 5.320
Epoch  11 Batch   13/33   train_loss = 5.492
Epoch  11 Batch   15/33   train_loss = 5.442
Epoch  11 Batch   17/33   train_loss = 5.462
Epoch  11 Batch   19/33   train_loss = 5.432
Epoch  11 Batch   21/33   train_loss = 5.545
Epoch  11 Batch   23/33   train_loss = 5.492
Epoch  11 Batch   25/33   train_loss = 5.414
Epoch  11 Batch   27/33   train_loss = 5.457
Epoch  11 Batch   29/33   train_loss = 5.428
Epoch  11 Batch   31/33   train_loss = 5.400
Epoch  12 Batch    0/33   train_loss = 5.323
Epoch  12 Batch    2/33   train_loss = 5.252
Epoch  12 Batch    4/33   train_loss = 5.317
Epoch  12 Batch    6/33   train_loss = 5.364
Epoch  12 Batch    8/33   train_loss = 5.382
Epoch  12 Batch   10/33   train_loss = 5.482
Epoch  12 Batch   12/33   train_loss = 5.380
Epoch  12 

Epoch  22 Batch    6/33   train_loss = 4.525
Epoch  22 Batch    8/33   train_loss = 4.526
Epoch  22 Batch   10/33   train_loss = 4.603
Epoch  22 Batch   12/33   train_loss = 4.555
Epoch  22 Batch   14/33   train_loss = 4.634
Epoch  22 Batch   16/33   train_loss = 4.497
Epoch  22 Batch   18/33   train_loss = 4.540
Epoch  22 Batch   20/33   train_loss = 4.511
Epoch  22 Batch   22/33   train_loss = 4.591
Epoch  22 Batch   24/33   train_loss = 4.523
Epoch  22 Batch   26/33   train_loss = 4.489
Epoch  22 Batch   28/33   train_loss = 4.560
Epoch  22 Batch   30/33   train_loss = 4.539
Epoch  22 Batch   32/33   train_loss = 4.600
Epoch  23 Batch    1/33   train_loss = 4.304
Epoch  23 Batch    3/33   train_loss = 4.373
Epoch  23 Batch    5/33   train_loss = 4.546
Epoch  23 Batch    7/33   train_loss = 4.503
Epoch  23 Batch    9/33   train_loss = 4.423
Epoch  23 Batch   11/33   train_loss = 4.344
Epoch  23 Batch   13/33   train_loss = 4.497
Epoch  23 Batch   15/33   train_loss = 4.432
Epoch  23 

Epoch  33 Batch    9/33   train_loss = 3.791
Epoch  33 Batch   11/33   train_loss = 3.703
Epoch  33 Batch   13/33   train_loss = 3.871
Epoch  33 Batch   15/33   train_loss = 3.794
Epoch  33 Batch   17/33   train_loss = 3.858
Epoch  33 Batch   19/33   train_loss = 3.762
Epoch  33 Batch   21/33   train_loss = 3.797
Epoch  33 Batch   23/33   train_loss = 3.838
Epoch  33 Batch   25/33   train_loss = 3.772
Epoch  33 Batch   27/33   train_loss = 3.879
Epoch  33 Batch   29/33   train_loss = 3.718
Epoch  33 Batch   31/33   train_loss = 3.734
Epoch  34 Batch    0/33   train_loss = 3.713
Epoch  34 Batch    2/33   train_loss = 3.674
Epoch  34 Batch    4/33   train_loss = 3.706
Epoch  34 Batch    6/33   train_loss = 3.726
Epoch  34 Batch    8/33   train_loss = 3.728
Epoch  34 Batch   10/33   train_loss = 3.772
Epoch  34 Batch   12/33   train_loss = 3.776
Epoch  34 Batch   14/33   train_loss = 3.807
Epoch  34 Batch   16/33   train_loss = 3.748
Epoch  34 Batch   18/33   train_loss = 3.754
Epoch  34 

Epoch  44 Batch   12/33   train_loss = 3.069
Epoch  44 Batch   14/33   train_loss = 3.072
Epoch  44 Batch   16/33   train_loss = 3.065
Epoch  44 Batch   18/33   train_loss = 3.058
Epoch  44 Batch   20/33   train_loss = 3.055
Epoch  44 Batch   22/33   train_loss = 3.045
Epoch  44 Batch   24/33   train_loss = 3.108
Epoch  44 Batch   26/33   train_loss = 3.020
Epoch  44 Batch   28/33   train_loss = 3.076
Epoch  44 Batch   30/33   train_loss = 3.106
Epoch  44 Batch   32/33   train_loss = 3.039
Epoch  45 Batch    1/33   train_loss = 2.865
Epoch  45 Batch    3/33   train_loss = 2.977
Epoch  45 Batch    5/33   train_loss = 3.068
Epoch  45 Batch    7/33   train_loss = 2.983
Epoch  45 Batch    9/33   train_loss = 3.032
Epoch  45 Batch   11/33   train_loss = 2.949
Epoch  45 Batch   13/33   train_loss = 3.054
Epoch  45 Batch   15/33   train_loss = 3.000
Epoch  45 Batch   17/33   train_loss = 3.069
Epoch  45 Batch   19/33   train_loss = 2.943
Epoch  45 Batch   21/33   train_loss = 3.007
Epoch  45 

...

Epoch 255 Batch    3/33   train_loss = 0.202
Epoch 255 Batch    5/33   train_loss = 0.190
Epoch 255 Batch    7/33   train_loss = 0.207
Epoch 255 Batch    9/33   train_loss = 0.217
Epoch 255 Batch   11/33   train_loss = 0.199
Epoch 255 Batch   13/33   train_loss = 0.197
Epoch 255 Batch   15/33   train_loss = 0.183
Epoch 255 Batch   17/33   train_loss = 0.211
Epoch 255 Batch   19/33   train_loss = 0.188
Epoch 255 Batch   21/33   train_loss = 0.211
Epoch 255 Batch   23/33   train_loss = 0.206
Epoch 255 Batch   25/33   train_loss = 0.195
Epoch 255 Batch   27/33   train_loss = 0.199
Epoch 255 Batch   29/33   train_loss = 0.186
Epoch 255 Batch   31/33   train_loss = 0.202
Epoch 256 Batch    0/33   train_loss = 0.195
Epoch 256 Batch    2/33   train_loss = 0.209
Epoch 256 Batch    4/33   train_loss = 0.203
Epoch 256 Batch    6/33   train_loss = 0.172
Epoch 256 Batch    8/33   train_loss = 0.214
Epoch 256 Batch   10/33   train_loss = 0.198
Epoch 256 Batch   12/33   train_loss = 0.217
Epoch 256 

Epoch 266 Batch    6/33   train_loss = 0.171
Epoch 266 Batch    8/33   train_loss = 0.214
Epoch 266 Batch   10/33   train_loss = 0.198
Epoch 266 Batch   12/33   train_loss = 0.216
Epoch 266 Batch   14/33   train_loss = 0.196
Epoch 266 Batch   16/33   train_loss = 0.224
Epoch 266 Batch   18/33   train_loss = 0.196
Epoch 266 Batch   20/33   train_loss = 0.192
Epoch 266 Batch   22/33   train_loss = 0.185
Epoch 266 Batch   24/33   train_loss = 0.220
Epoch 266 Batch   26/33   train_loss = 0.196
Epoch 266 Batch   28/33   train_loss = 0.175
Epoch 266 Batch   30/33   train_loss = 0.187
Epoch 266 Batch   32/33   train_loss = 0.203
Epoch 267 Batch    1/33   train_loss = 0.180
Epoch 267 Batch    3/33   train_loss = 0.202
Epoch 267 Batch    5/33   train_loss = 0.190
Epoch 267 Batch    7/33   train_loss = 0.206
Epoch 267 Batch    9/33   train_loss = 0.216
Epoch 267 Batch   11/33   train_loss = 0.199
Epoch 267 Batch   13/33   train_loss = 0.196
Epoch 267 Batch   15/33   train_loss = 0.182
Epoch 267 

Epoch 277 Batch    9/33   train_loss = 0.216
Epoch 277 Batch   11/33   train_loss = 0.198
Epoch 277 Batch   13/33   train_loss = 0.196
Epoch 277 Batch   15/33   train_loss = 0.182
Epoch 277 Batch   17/33   train_loss = 0.210
Epoch 277 Batch   19/33   train_loss = 0.188
Epoch 277 Batch   21/33   train_loss = 0.210
Epoch 277 Batch   23/33   train_loss = 0.205
Epoch 277 Batch   25/33   train_loss = 0.194
Epoch 277 Batch   27/33   train_loss = 0.198
Epoch 277 Batch   29/33   train_loss = 0.185
Epoch 277 Batch   31/33   train_loss = 0.201
Epoch 278 Batch    0/33   train_loss = 0.194
Epoch 278 Batch    2/33   train_loss = 0.208
Epoch 278 Batch    4/33   train_loss = 0.203
Epoch 278 Batch    6/33   train_loss = 0.171
Epoch 278 Batch    8/33   train_loss = 0.213
Epoch 278 Batch   10/33   train_loss = 0.197
Epoch 278 Batch   12/33   train_loss = 0.216
Epoch 278 Batch   14/33   train_loss = 0.195
Epoch 278 Batch   16/33   train_loss = 0.223
Epoch 278 Batch   18/33   train_loss = 0.196
Epoch 278 

Epoch 288 Batch   12/33   train_loss = 0.216
Epoch 288 Batch   14/33   train_loss = 0.195
Epoch 288 Batch   16/33   train_loss = 0.223
Epoch 288 Batch   18/33   train_loss = 0.196
Epoch 288 Batch   20/33   train_loss = 0.191
Epoch 288 Batch   22/33   train_loss = 0.184
Epoch 288 Batch   24/33   train_loss = 0.219
Epoch 288 Batch   26/33   train_loss = 0.196
Epoch 288 Batch   28/33   train_loss = 0.174
Epoch 288 Batch   30/33   train_loss = 0.186
Epoch 288 Batch   32/33   train_loss = 0.202
Epoch 289 Batch    1/33   train_loss = 0.179
Epoch 289 Batch    3/33   train_loss = 0.201
Epoch 289 Batch    5/33   train_loss = 0.189
Epoch 289 Batch    7/33   train_loss = 0.206
Epoch 289 Batch    9/33   train_loss = 0.215
Epoch 289 Batch   11/33   train_loss = 0.198
Epoch 289 Batch   13/33   train_loss = 0.195
Epoch 289 Batch   15/33   train_loss = 0.182
Epoch 289 Batch   17/33   train_loss = 0.210
Epoch 289 Batch   19/33   train_loss = 0.187
Epoch 289 Batch   21/33   train_loss = 0.210
Epoch 289 

Epoch 299 Batch   15/33   train_loss = 0.181
Epoch 299 Batch   17/33   train_loss = 0.209
Epoch 299 Batch   19/33   train_loss = 0.187
Epoch 299 Batch   21/33   train_loss = 0.210
Epoch 299 Batch   23/33   train_loss = 0.205
Epoch 299 Batch   25/33   train_loss = 0.194
Epoch 299 Batch   27/33   train_loss = 0.197
Epoch 299 Batch   29/33   train_loss = 0.184
Epoch 299 Batch   31/33   train_loss = 0.200
Model Trained and Saved


## Save Parameters
Save `seq_length` and `save_dir` for generating a new TV script.

In [27]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params((seq_length, save_dir))

# Checkpoint

In [28]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
seq_length, load_dir = helper.load_params()

## Implement Generate Functions
### Get Tensors
Get tensors from `loaded_graph` using the function [`get_tensor_by_name()`](https://www.tensorflow.org/api_docs/python/tf/Graph#get_tensor_by_name).  Get the tensors using the following names:
- "input:0"
- "initial_state:0"
- "final_state:0"
- "probs:0"

Return the tensors in the following tuple `(InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)` 

In [29]:
def get_tensors(loaded_graph):
    """
    Get input, initial state, final state, and probabilities tensor from <loaded_graph>
    :param loaded_graph: TensorFlow graph loaded from file
    :return: Tuple (InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)
    """
    # TODO: Implement Function
    _input = loaded_graph.get_tensor_by_name("input:0")
    _initial_state = loaded_graph.get_tensor_by_name("initial_state:0")
    _final_state = loaded_graph.get_tensor_by_name("final_state:0")
    _probs = loaded_graph.get_tensor_by_name("probs:0")
    return (_input, _initial_state, _final_state, _probs)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_tensors(get_tensors)

Tests Passed


### Choose Word
Implement the `pick_word()` function to select the next word using `probabilities`.

In [30]:
def pick_word(probabilities, int_to_vocab):
    """
    Pick the next word in the generated text
    :param probabilities: Probabilites of the next word
    :param int_to_vocab: Dictionary of word ids as the keys and words as the values
    :return: String of the predicted word
    """
    # TODO: Implement Function
    idx = np.argmax(probabilities)
    word = int_to_vocab[idx]
    return word


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_pick_word(pick_word)

Tests Passed


## Generate TV Script
This will generate the TV script for you.  Set `gen_length` to the length of TV script you want to generate.

In [31]:
gen_length = 1000
# homer_simpson, moe_szyslak, or Barney_Gumble
prime_word = 'moe_szyslak'

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_dir + '.meta')
    loader.restore(sess, load_dir)

    # Get Tensors from loaded model
    input_text, initial_state, final_state, probs = get_tensors(loaded_graph)

    # Sentences generation setup
    gen_sentences = [prime_word + ':']
    prev_state = sess.run(initial_state, {input_text: np.array([[1]])})

    # Generate sentences
    for n in range(gen_length):
        # Dynamic Input
        dyn_input = [[vocab_to_int[word] for word in gen_sentences[-seq_length:]]]
        dyn_seq_length = len(dyn_input[0])

        # Get Prediction
        probabilities, prev_state = sess.run(
            [probs, final_state],
            {input_text: dyn_input, initial_state: prev_state})
        
        pred_word = pick_word(probabilities[0,dyn_seq_length-1,:], int_to_vocab)

        gen_sentences.append(pred_word)
    
    # Remove tokens
    tv_script = ' '.join(gen_sentences)
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        tv_script = tv_script.replace(' ' + token.lower(), key)
    tv_script = tv_script.replace('\n ', '\n')
    tv_script = tv_script.replace('( ', '(')
        
    print(tv_script)

INFO:tensorflow:Restoring parameters from ./save
moe_szyslak:(sadly) oh, homer.


smokin'_joe_frazier: keep those pickled eggs comin'?.(sobs)
homer_simpson:(tipsy) gee, sorry, okay.
homer_simpson:(amid curious tell it, uh, homer's pants are give this beer in your money.
moe_szyslak: nah, i can't really say that?
lenny_leonard:(drunk) hey, i gotta be in the bar.
homer_simpson: how much for my new money?
moe_szyslak: yeah, you can't my name, moe? but that was all a little girl.
moe_szyslak: uh, i'll send a" love) that, okay, and they drove here.
homer_simpson:(indignant) gee, you can live in that drink are too bad beer that alcohol about my beer.
homer_simpson: i am. it was andy.
moe_szyslak:(anguished) why, i can't believe it, moe.
homer_simpson: hey, moe, can i say this delicately?
homer_simpson:(sunk) i thought i can use my new money.
moe_szyslak: yeah, that's right now that--
homer_simpson:(to moe) oh, no, that's sweet.(guy) hey, you wanna send a drink?(laughs)... well, i can't belie

# The TV Script is Nonsensical
It's ok if the TV script doesn't make any sense.  We trained on less than a megabyte of text.  In order to get good results, you'll have to use a smaller vocabulary or get more data.  Luckly there's more data!  As we mentioned in the begging of this project, this is a subset of [another dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data).  We didn't have you train on all the data, because that would take too long.  However, you are free to train your neural network on all the data.  After you complete the project, of course.
# Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_tv_script_generation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.